In [ ]:
# Create partition for given images to train/val/test groups
import random
import re
import shutil
from concurrent.futures import ThreadPoolExecutor
from pathlib import Path

from tqdm import tqdm

# Specify the directories
raw_data_folder = Path("/home/idies/workspace/ssec_neural_morphology/SimulationsNew_Tif3334/images/")
processed_folder = Path("/home/idies/workspace/ssec_neural_morphology/SimulationsNew_Tif3334/neuro-morpho/")


def process_file(image_name, raw_data_folder, processed_folder):
    label_name = raw_data_folder / re.sub(r"Realistic-SBR-.", "Skeleton", image_name.name)
    p = random.random()

    if p <= 0.6:  # train
        subfolder = "train"
    elif p <= 0.8:  # validation
        subfolder = "val"
    else:  # test
        subfolder = "test"

    dest_img = processed_folder / subfolder / "imgs" / image_name.name
    new_label_name = processed_folder / subfolder / "lbls" / image_name.name.replace("Realistic-SBR-", "Skeleton-")

    shutil.copy(image_name, dest_img)
    shutil.copy(label_name, new_label_name)


def organize_files(raw_data_folder, processed_folder, max_workers=8):
    # List all image files in the raw_data_folder
    image_files = sorted(list(Path(raw_data_folder).glob("Realistic-SBR-*")))

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        list(
            tqdm(
                executor.map(lambda img: process_file(img, raw_data_folder, processed_folder), image_files),
                total=len(image_files),
                desc="Copying files",
            )
        )


# Example call:
organize_files(raw_data_folder, processed_folder)